# Tema 2. Análisis de secuencias

## Análisis de calidad de secuencias cortas

Las secuencias cortas que usaremos en esta práctica provienen de un experimento de secuenciación de un aisalado de coronavirus. Se encuentran en los archivos `data/ERR4423464_1.fastq` y `data/ERR4423464_2.fastq`, que son archivos de texto con formato **fastq**. Hay dos porque se trata de lecturas emparejadas (*paired end*): cada fragmento original de DNA ha sido secuenciado primero desde un extremo y después desde el opuesto. Las dos lecturas pueden llegar a solaparse, si el fragmento original no era demasiado largo. Primero, échale un vistazo al archivo. Deberías poder observar algo así:

    @ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=109
    CTGCTACACGCTGCGAAGCTCCCAATTTGTAATAAGAAAGCGTTCGTGATGTAGCAACAGTGATTTCTTTAGGC...
    +ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=109
    BC@CCDE9F-@C7C+++6@CEFF8,CFFCFA9F9,,,,,,C:FFFCF@,C9F99C,6C8<C9,EFFFFEE9,,9...


Puedes aprender más sobre el formato **fastq** [aquí](https://en.wikipedia.org/wiki/FASTQ_format). ¿Puedes identificar las diferentes partes del formato? ¿Para qué sirve la cuarta línea de cada registro?

A continuación utilizaremos el paquete *ShortReads* de R para comprobar la calidad y características de las secuencias. Este paquete ha sido creado por Martin Morgan, Michael Lawrence, Simon Anders, y distribuido con licencia *Artistic-2.0* en [Bioconductor](http://www.bioconductor.org/packages/release/bioc/html/ShortRead.html).

Recuerda que para ejecutar el código mostrado a continuación (o aquel que tú escribas) debes seleccionar el bloque de código y pinchar en el botón de *Run*, en el menú superior. El código está distribuido en diferentes bloques para facilitar la comprensión de cada paso.

In [ ]:
# Para poder usar funciones de análisis de secuencias cortas
# necesitamos cargar algun paquete específico, como "ShortReads":
suppressMessages(library('ShortRead', logical.return = TRUE))

# library('ShortRead')  # haría lo mismo, pero con un largo mensaje.

In [ ]:
# La funicón qa() aplica un análisis estándard a los archivos indicados,
# en este caso, los únicos dos archivos con extensión "fastq" en la carpeta
# "data":

ResumenCalidad <- qa('data', type = 'fastq', pattern = '*.fastq')

In [ ]:
# Podemos ver el contenido del objeto "ResumenCalidad" así:

ResumenCalidad

In [ ]:
# Así como acceder a sus elementos individuales:

ResumenCalidad[['readCounts']]
# Edita este bloque para mostrar otros elementos del "ResumenCalidad",
# como por ejemplo "baseCalls".

In [ ]:
# Para mostrar sólo una pequeña parte de los elementos más grandes,
# puedes usar las funciones head() y tail():
head(ResumenCalidad[['baseQuality']])

In [ ]:
# Por último, podemos generar un archivo html con toda la información
# generada por la función qa().
report(ResumenCalidad, dest = 'ResumenCalidad.html')

Después de experimentar con los resultados de la función *qa()* del paquete *ShortRead*, habrás observado que en algunos aspectos no se ajusta a nuestros datos. Por ejemplo, las secuencias en un archivo **fastq** no están alineadas, y por tanto *qa()* no ha podido informar sobre la calidad de los alineamientos. Además, un análisis estándard puede no responder a todas nuestras necesidades.

Como las longitudes de las secuencias no parecen estar registradas en el informe de *qa()*, es necesario leer de nuevo los datos originales. Para ello, disponemos de diferentes funciones. Normalmente, para la finalidad de comprobar la calidad de los datos, se usa solo una fracción aleatoria de las secuencias. Para eso sirve la función `FastqSampler()`. Pero en este caso particular se trata de dos archivos relativamente pequeños, y podemos usar la función `readFastq()`, que cargará en memoria la totalidad de las secuencias.

In [ ]:
lecturas.1 <- readFastq('data/ERR4423464_1.fastq')
lecturas.2 <- readFastq('data/ERR4423464_2.fastq')

# Para mostrar las primeras secuencias de cada muestra:
head(sread(lecturas.1))

In [ ]:
# Podemos mostrar también, sus calidades:
head(quality(lecturas.1))

In [ ]:
# Y extraer directamente sus longitudes:
head(width(lecturas.1))

In [ ]:
# Y para hacer un histograma, podemos usar la función hist():
hist(width(lecturas.1))

Intenta usar la función `mean()` para calcular la longitud media de las secuencias de cada archivo.

El análisis de la calidad de las secuencias obtenidas de un experimento de secuenciación es un paso fundamental en el que deben eliminarse las secuencias de baja calidad, o recortarse si en algun extremo la secuenciación a producido resultados pobres.

## Mapeo de lecturas cortas
El siguiente paso es normalmente el de identificar la posición de origen, en un genoma de referencia, de todas las secuencias cortas obtenidas por secuenciación. Existen muchos programas disponibles para ello. En el entorno de R podemos utilizar una implementación de Bowtie-2, llamada *Rbowtie2*, creada por Zheng Wei y Wei Zhang y disponible en [Bioconductor](https://bioconductor.org/packages/release/bioc/html/Rbowtie2.html) bajo licencia *GPL (>= 3)*.

El genoma de referencia es el del SARS-CoV-2, y se encuentra en el archivo `data/referencia.fna`. Éstas son sus primeras líneas:

    >NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
    ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
    CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
    TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
    TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
    CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
    GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
    CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT

¿Reconoces el formato? Antes de *mapear* o alinear las secuencias cortas al genoma de referencia es necesario preparar (de hecho, *indexar*) el genoma de referencia. El índice se compondrá de unos pocos archivos, con el mismo nombre base y diferentes terminaciones. Para tener los archivos ordenados, creamos una carpeta donde guardar el índice.

In [ ]:
suppressMessages(library(Rbowtie2))
dir.create('indice')

# Observa como podemos usar más de una línea para escribir una orden más clara:
build_output <- bowtie2_build(references = 'data/referencia.fna',
                              bt2Index = 'indice/SARS-Cov2',
                              '--quiet',
                              overwrite = TRUE)

# En esta variable hemos guardado los mensajes de salida del proceso. Veamos si hay
# algun error:
build_output

La construcción del índice del genoma de referencia acepta muchas opciones. Las únicas que son imprescindibles son la dirección del archivo donde está el genoma de referencia y el nombre base que queremos darles a los archivos del índice. Comprueba que el índice ha sido creado en la carpeta designada. Los archivos que lo componen no son archivos de texto.

Ahora podemos mapear las secuencias cortas al genoma del coronavirus SARS-Cov-2. Si has dado un nombre diferente a la carpeta del índice, no olvides cambiarlo aquí también. También puedes elegir el nombre del archivo donde se guardarán los resultados en formato SAM.

In [ ]:
mapeo_output <- bowtie2(bt2Index = 'indice/SARS-Cov2',
                        samOutput = 'ERR4423464.sam',
                        seq1 = 'data/ERR4423464_1.fastq',
                        seq2 = 'data/ERR4423464_2.fastq',
                        overwrite = TRUE)
mapeo_output

Dedica un tiempo a leer el mensaje de salida del mapeo. ¿Te parece un resultado satisfactorio? ¿Puedes interpretar toda la información?

El archivo de salida, `ERR4423464.sam`, sí es un archivo de texto en formato SAM, que puedes abrir para explorarlo. Observarás que las líneas son más largas que el espacio disponible en la pantalla, lo que dificulta reconocer la regularidad del formato. Para identificar mejor las columnas puedes usar el comando siguiente en una shell de Unix:

`less -S ERR4423464.sam`

Visita [este enlace](https://en.wikipedia.org/wiki/SAM_(file_format)) o bien la [publicación original](https://academic.oup.com/bioinformatics/article/25/16/2078/204688) donde se describe este formato. De todas formas, es más práctico (y ocupa menos) la versión *binaria* del formato SAM, llamada *BAM*. Vamos a crear una versión *bam* de nuestros alineamientos para usarla en el próximo paso:

In [ ]:
suppressMessages(library(Rsamtools, quietly = TRUE))
asBam('ERR4423464.sam', destination='ERR4423464', overwrite = TRUE)

# Esto nos ofrece un resumen de los tipos de alineamientos encontrados:
quickBamFlagSummary('ERR4423464.bam')

El formato SAM contiene mucha información pero no nos permite una visión panorámica a lo largo del genoma de referencia. Para poder visualizar cómo se distribuyen las secuencias cortas a lo largo del genoma de referencia, necesitamos *amontonarlas*. La función `pileup()` permite contar cuantas veces ha sido *leída* cada posición (es decir, su **cobertura**), además de poder registrar qué nucleótidos diferentes se han encontrado en cada posición y con qué frecuencia cada uno, entre otras cosas. Nota en el código a continuación que podemos elegir la calidad mínima de una base leída para que la contemos en su cobertura

In [ ]:
cobertura <- pileup('ERR4423464.bam',
                    pileupParam = PileupParam(max_depth = 10000,
                                              min_base_quality = 13,
                                              min_mapq = 13,
                                              min_nucleotide_depth = 1,
                                              min_minor_allele_depth = 0,
                                              distinguish_strands = FALSE,
                                              distinguish_nucleotides = FALSE))
head(cobertura)

Antes de ver cómo varía la cobertura a lo largo del genoma en este experimento de secuenciación, consideremos qué cabe esperar. Hemos usado 65.135 pares de secuencias cortas, cuya longitud media estaba alrededor de 154. Aproximadamente esperaríamos unas $65.135 \times 2 \times 154 = 20.061.580$ bases secuenciadas. Como el genoma de coronavirus SARS-Cov-2 mide unas 30.000 bases, en principio tenemos suficientes secuencias cortas como para cubrir el genoma de referencia $20.061.580/30000=669$ veces. En la gráfica que generaremos a continuación la línea roja marcará la cobertura media obtenida (676) y la azul, la mediana.

In [ ]:
# Los dos primeros argumentos de la función plot() son las coordenadas
# 
plot(cobertura$pos, cobertura$count, type = 'l',
     xlab = 'Posición', ylab = 'Cobertura')
abline(h = mean(cobertura$count),   col = 'red')
abline(h = median(cobertura$count), col = 'blue')

Si la secuenciación hubiera sido realmente aleatoria, de manera que todas las posiciones tuvieran la misma probabilidad de ser secuenciadas, la cobertura seguiría una distribución de Poisson, donde la media es igual a la varianza. En este caso la varianza (calcúlala con `var(cobertura$count)`), es mucho mayor que la media. ¿A qué se te ocurre que podría deberse el hecho de que algunas posiciones hayan sido secuenciadas tanto más que otras?

## Detección de variantes

El objetivo de muchos proyectos de secuenciación es detectar y analizar las posiciones (y fragmentos) variables, en los que no todos los individuos secuenciados tienen la misma información genética. Un tipo básico de variación son las mutaciones puntuales o SNPs. Diferenciar las posiciones variables de las que no lo son es un problema de clasificación que puede resultar difícil debido a que las secuencias cortas incluyen errores de secuenciación. 

Fuera del entorno de R, existen muchos otros programas para detectar variantes. [GATK](https://gatk.broadinstitute.org/hc/en-us), desarrollada por el Broad Institute, es una de las herramientas más populares para la detección de variantes a gran escala. Se distribuye como *software* libre, bajo licencia BSD modificada de 3 cláusulas. Otro programa interesante es [Freebayes](https://github.com/freebayes/freebayes), distribuido bajo licencia MIT.

Sin entrar en los detalles del proceso, el ejemplo básico siguiente nos da una idea de los pasos a seguir en R y nos permitirá observar el formato **VCF** de los resultados. Vamos a usar los paquetes [VariantTools](https://bioconductor.org/packages/release/bioc/html/VariantTools.html) y [gmapR](https://bioconductor.org/packages/release/bioc/html/gmapR.html).

El primer paso es indexar de nuevo el genoma de referencia. El paquete `gmapR` es independiente de `Rbowtie2` y no usa el mismo tipo de índice. La función `GmapGenome()` puede crear un índice a partir de un archivo **fasta**, siempre que se le indique la dirección (o *path*) de ese archivo de la forma adecuada. Así pues, antes de crear el índice, creamos la variable que indica dónde está la secuencia del genoma de referencia en formato **fasta**.
 

In [ ]:
suppressMessages(library('VariantTools'))
suppressMessages(library('gmapR'))

# Creamos un objeto 'FastaFile' a partir del archivo data/referencia.fna
referencia.fasta  <- rtracklayer::FastaFile(
    paste(getwd(), 'data/referencia.fna', sep = '/'))

In [ ]:
# Aquí se indexa el genoma de referencia.
referencia.genoma <- GmapGenome(referencia.fasta, create = TRUE)

Esta expresión habrá generado un error. Léelo e intenta descifrar su significado. Ante situaciones como esta, es recomendable buscar en internet el texto o las palabras clave del error. En este caso, el problema es que la máquina virtual donde se ejecutan estas órdenes no dispone de suficiente memoria RAM para generar los archivos del índice. Para solucionarlo, utilizaremos un índice ya creado y contenido en el archivo `data/gmap.tar.gz`. Antes de utilizarlo habrá que desempaquetarlo.

In [ ]:
untar('data/gmap.tar.gz')

Y ahora podemos informar a la función `GmapGenome()` de la ubicación del índice que acabamos de desempaquetar. Observaràs que la forma de hacerlo es bastante enrevesada, porque al argumento `directory` se le asigna el resultado de combinar tres funciones diferentes:

In [ ]:
referencia.genoma <- GmapGenome(genome = 'referencia',
                               directory = GmapGenomeDirectory(file.path(getwd(),'gmap')))

Ahora sí tenemos el genoma de referencia en el formato adecuado y estamos en condiciones de usarlo, junto con los alineamientos del archivo `ERR4423464.bam` para detectar variantes.

La función `TallyVariantsParam()` permite ajustar varios parámetros que afectan la sensibilidad y la especificidad del proceso de detección de variantes. Puedes averiguar qué parámetros son esos leyendo la ayuda de la función: `help(TallyVariantsParam)`. Si no especificamos más que el genoma de referencia, aceptamos los valores por defecto.

In [ ]:
parametros <- TallyVariantsParam(referencia.genoma)

Normalmente se detectan variantes entre un conjunto de muestras o individuos secuenciados. En este ejercicio estamos usando sólo una muestra, representada por el archivo `ERR4423464.bam`. Aquí, las *variantes* que buscamos son diferencias entre el genoma de referencia y el de la muestra secuenciada. 

In [ ]:
preliminar <- callVariants('ERR4423464.bam', parametros)
summary(preliminar)

Las posiciones variables estan indicadas como *rangos*, pues podrían abarcar más de un nucleótido. Siempre, el resultado de detección de variantes es susceptible de contener falsos positivos. El paso siguiente consiste en filtrar los resultados para quedarnos solo con las variantes más probablemente verdaderas.

In [ ]:
filtradas  <- postFilterVariants(preliminar)
summary(filtradas)

¿Cuántas posiciones variables hemos detectado? ¿Consigues extraer e interpretar la información contenida en el objeto `filtradas`? Existen muchas funciones para trabajar con esta clase de objeto, pero aprenderlas no es nuestro objetivo ahora. Tal vez sea más fácil y práctico guardar la información sobre las variantes filtradas en un archivo de texto con formato VCF.

El formato VCF necesita los nombres de las muestras (en nuestro caso solo una), que normalmente coinciden con los nombres de los archivos `.bam`. Como esta información no está contenida todavía en el objeto `filtradas`, la introducimos ahora:

In [ ]:
sampleNames(filtradas) <- 'ERR4423464'

Y finalmente creamos el archivo **VCF**:

In [ ]:
writeVcf(filtradas, filename = 'ERR4423464.vcf')

El formato **VCF** está descrito en [este documento](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwicz7ih05zuAhUHExoKHQB-DY8QFjAIegQIARAC&url=https%3A%2F%2Fsamtools.github.io%2Fhts-specs%2FVCFv4.2.pdf&usg=AOvVaw3UrlHdXnBVzm0df9OE90Rm). Al abrir el archivo `ERR4423464.vcf` con un editor de texto, normalmente las líneas más largas que la pantalla acaban ocupando más de una línea, lo que dificulta la identificación del formato. Para poder observar mejor el formato del archivo es recomendable abrirlo, por ejemplo, desde una terminal con el comando:

`less -S ERR4423464.vcf`